# Validation


## Imports / Setup


In [1]:
import boto3
import pandas as pd
import json
import os
from pathlib import Path
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
from ladybug.epw import EPW
from lightning.pytorch import Trainer
import torch
import numpy as np

from ml.surrogate import Surrogate
from ml.predict import predict_ubem
from utils.constants import JOULES_TO_KWH
from weather.weather import extract

from archetypal import settings

if os.name == "nt":
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("C:/EnergyPlusV22-2-0")
else:
    settings.ep_version == "22.2.0"
    settings.energyplus_location = Path("/usr/local/EnergyPlus-22-2-0")

from archetypal.idfclass import IDF

Couldn't import dot_parser, loading of dot files will not be possible.


c:\tools\Anaconda3\envs\ml-for-bem-final\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package energy-pandas is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Config Vars


In [2]:
bucket = "ml-for-bem"
s3 = boto3.client("s3")

local_dir = Path("data") / "temp"
experiment_name = "validation/v2/seattle"
should_gather_idf = False

registry = "ml-for-building-energy-modeling/model-registry"
model = "Global UBEM Shoebox Surrogate with Combined TS Embedder"
tag = "v3"
resource = "model.ckpt"

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## Helper Fns


In [19]:
# list objects in bucket using paginator
def get_files_in_experiment(experiment):
    print("Listing files for experiment: ", experiment, "...")
    files = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=experiment):
        if page.get("Contents"):
            for obj in page["Contents"]:
                files.append(obj["Key"])
    return files


def download(file):
    filepath = local_dir / file
    os.makedirs(filepath.parent, exist_ok=True)
    s3.download_file(bucket, file, filepath)
    return filepath


def download_and_open_json(file):
    filepath = download(file)
    with open(filepath, "r") as f:
        return json.load(f)


def download_and_open_hdf(file):
    filepath = download(file)
    return pd.read_hdf(filepath)


def get_weather():
    weather_folder = experiment_name + "/epw"
    # list files in weather folder
    weather_files = get_files_in_experiment(weather_folder)
    with ThreadPoolExecutor(max_workers=8) as executor:
        weather_files = list(
            tqdm(executor.map(download, weather_files), total=len(weather_files))
        )
    return weather_files


def download_and_open_ground_truth():
    file = experiment_name + "/Buildings.hdf"
    true_df = download_and_open_hdf(file)

    true_df = true_df.sort_index(level="file_name")
    building_id = true_df.index.to_frame()["file_name"].str.split(".").str[0]
    building_id.name = "building_id"
    true_df = true_df.set_index(building_id, append=True).sort_index(
        level="building_id"
    )

    true_df = true_df * JOULES_TO_KWH
    return true_df

## Gather/Download Validation Files


In [25]:
if should_gather_idf:
    idf_files = get_files_in_experiment(experiment_name + "/idf")
    idf_files = idf_files[:100]
    with ThreadPoolExecutor(max_workers=8) as executor:
        idf_files = list(tqdm(executor.map(download, idf_files), total=len(idf_files)))

features_fp = Path(experiment_name).parent / "features.hdf"
features_fp = download(features_fp.as_posix())
features = pd.read_hdf(features_fp, key="shoeboxes")
building_meta = pd.read_hdf(features_fp, key="buildings")

In [27]:
len(features), len(building_meta)

(216940, 9451)

## Check for errors in hdf etc


In [5]:
def test_idf(building_id):
    idf_path = (
        Path("data")
        / "temp"
        / Path(experiment_name).parent
        / "idf"
        / f"{building_id:09d}.idf"
    ).as_posix()
    idf = IDF(idf_path)
    zones = idf.idfobjects["ZONE"]
    # remove final _<number> from zone names
    zone_names = ["_".join(zone.Name.split("_")[:-1]) for zone in zones]
    zones_by_name = {zone_name: zone for zone_name, zone in zip(zone_names, zones)}

    area = 0
    for zone in zones_by_name.values():
        area += zone.Floor_Area

    assert abs(building_meta.loc[building_id, "gfa"] - area) < 1
    areas = (
        features.groupby(level="building_id")
        .sum()
        .loc[building_id, ["core_area", "zone_perimeter_area"]]
    )
    assert np.abs(areas["core_area"] / 4 + areas["zone_perimeter_area"] - area) < 1e-1


for building_id in tqdm(building_meta.index.to_list()[:10]):
    test_idf(building_id)

  0%|          | 0/10 [00:00<?, ?it/s]

## Allocate Shoebox dimensions

Floor-to-floor height and shoebox width are fixed.


In [6]:
# sb_perim_facade_area = features.width * features.height
# sb_perim_area = features.zone_perim_floor_area_to_facade_area * sb_perim_facade_area
# sb_perim_depth = sb_perim_area / features.width
# sb_core_depth = features.zone_core_area_to_perimeter_area * sb_perim_depth

features["perim_depth"] = features.building_computed_perim_depth
features["core_depth"] = features.building_computed_core_depth

## Load Ground Truth


In [20]:
true_results = download_and_open_ground_truth()

In [23]:
len(building_meta.index.get_level_values("building_id"))

9451

In [24]:
len(true_results.index.get_level_values("building_id"))

9746

In [15]:
true_results = true_results.loc[
    (slice(None), slice(None), slice(None), building_meta.index), :
]

assert (
    true_results.index.get_level_values("building_id") == building_meta.index.values
).all()

45

In [16]:
true_results = true_results / building_meta.gfa.values.reshape(-1, 1)

## Load Weather, Schedules


In [15]:
city = "seattle"
weather_files = get_weather(city)
epw_path = weather_files[0]
epw = EPW(epw_path)
climate = extract(epw)
scheds = np.load("data/schedules.npy")

Listing files for experiment:  idf/batch/validation_v1/seattle/epw ...


  0%|          | 0/1 [00:00<?, ?it/s]

## Load Surrogate and Predict


In [20]:
surrogate = Surrogate.load_from_registry(registry, model, tag, resource)
surrogate.model.eval()
torch.set_float32_matmul_precision("medium")

trainer = Trainer(
    accelerator="auto",
    devices="auto",
    strategy="auto",
    enable_progress_bar=True,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
ml_shoebox_results, ml_monthly, ml_annual = predict_ubem(
    trainer=trainer,
    surrogate=surrogate,
    features=features.reset_index(),
    schedules=scheds,
    climate=climate,
    apply_cops=False,
    batch_size=64,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:02<00:00,  7.92it/s]


## Plotting


In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 15))
ml_ax = ax[0, 0]
idf_ax = ax[0, 1]
scatter_h_ax = ax[1, 0]
scatter_c_ax = ax[1, 1]


ml_ax.plot(ml_monthly.Heating.values.T, color="red", alpha=0.2)


ml_ax.plot(ml_monthly.Cooling.values.T, color="blue", alpha=0.2)
idf_ax.plot(true_results["Heating:DistrictHeating"].values.T, color="red", alpha=0.2)
idf_ax.plot(true_results["Cooling:DistrictCooling"].values.T, color="blue", alpha=0.2)
ml_ax.set_ylim(0, 100)
idf_ax.set_ylim(0, 100)
ml_ax.set_title("ML")
idf_ax.set_title("IDF")
scatter_h_ax.set_title("Heating Model Fit")
scatter_h_ax.set_xlabel("IDF")
scatter_h_ax.set_ylabel("ML")
scatter_c_ax.set_title("Cooling Model Fit")
scatter_c_ax.set_xlabel("IDF")
scatter_c_ax.set_ylabel("ML")
scatter_h_ax.scatter(
    true_results["Heating:DistrictHeating"].values.flatten(),
    ml_monthly.Heating.values.flatten().clip(0, 1000),
    color="red",
    alpha=0.2,
)
scatter_c_ax.scatter(
    true_results["Cooling:DistrictCooling"].values.flatten(),
    ml_monthly.Cooling.values.flatten().clip(0, 1000),
    color="blue",
    alpha=0.2,
)
# plot identity line on top of scatter
x = np.linspace(0, np.max(ml_monthly.Cooling.values.flatten()), 100)
scatter_c_ax.plot(x, x, color="black", linestyle="--")
x = np.linspace(0, np.max(ml_monthly.Heating.values.flatten()), 100)
scatter_h_ax.plot(x, x, color="black", linestyle="--")
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
plt.suptitle("Annual Model Fits")
h_ax = ax[0]
c_ax = ax[1]
h_ax.set_title("Heating")
c_ax.set_title("Cooling")

h_ax.scatter(
    true_results["Heating:DistrictHeating"].sum(axis=1),
    ml_annual.Heating,
    color="red",
    alpha=0.2,
)
h_ax.set_ylim(0, 50)
h_ax.set_xlim(0, 50)

c_ax.scatter(
    true_results["Cooling:DistrictCooling"].sum(axis=1),
    ml_annual.Cooling,
    color="blue",
    alpha=0.2,
)
c_ax.set_ylim(0, 50)
c_ax.set_xlim(0, 50)